# **Adversarial Transferability Analysis using Adversarial Attacks**
# **Pretrain=False, 10 epoch**

## **1. Import Required Libraries**

In [1]:

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchattacks
import time
import pandas as pd
from torchvision import models

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')


Using device: cpu


## **2. Load CIFAR-10 Dataset**

In [2]:

# Load CIFAR-10 dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Standardizing input size for all models
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)


## **3. Define the Models (VGG-16, ResNet18, EfficientNet-B0)**

In [3]:

# Define VGG-16 model (Model A)
class VGGNet(nn.Module):
    def __init__(self):
        super(VGGNet, self).__init__()
        self.model = models.vgg16(pretrained=False)
        num_ftrs = self.model.classifier[6].in_features
        self.model.classifier[6] = nn.Linear(num_ftrs, 10)

    def forward(self, x):
        return self.model(x)

# Define ResNet18 model (Model B)
class ResNetModel(nn.Module):
    def __init__(self):
        super(ResNetModel, self).__init__()
        self.model = models.resnet18(pretrained=False)
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, 10)

    def forward(self, x):
        return self.model(x)

# Define EfficientNet-B0 model (Model C)
class EfficientNetModel(nn.Module):
    def __init__(self):
        super(EfficientNetModel, self).__init__()
        self.model = models.efficientnet_b0(pretrained=False)
        num_ftrs = self.model.classifier[1].in_features
        self.model.classifier[1] = nn.Linear(num_ftrs, 10)

    def forward(self, x):
        return self.model(x)

# Initialize models
model_A = VGGNet().to(device)
model_B = ResNetModel().to(device)
model_C = EfficientNetModel().to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer_A = optim.Adam(model_A.parameters(), lr=0.001)
optimizer_B = optim.Adam(model_B.parameters(), lr=0.001)
optimizer_C = optim.Adam(model_C.parameters(), lr=0.001)


c:\Users\cheey\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\cheey\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


## **4. Train the Models**

In [27]:

# Training function
def train_model(model, optimizer, model_name, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        print(f"{model_name} - Epoch {epoch+1}, Loss: {running_loss/len(trainloader):.4f}")

print("Training VGG-16...")
train_model(model_A, optimizer_A, "VGG-16")

print("Training ResNet18...")
train_model(model_B, optimizer_B, "ResNet18")

print("Training EfficientNet-B0...")
train_model(model_C, optimizer_C, "EfficientNet-B0")


Training VGG-16...
VGG-16 - Epoch 1, Loss: 2.3039
VGG-16 - Epoch 2, Loss: 2.3028
VGG-16 - Epoch 3, Loss: 2.3028
VGG-16 - Epoch 4, Loss: 2.3028
VGG-16 - Epoch 5, Loss: 2.3028
VGG-16 - Epoch 6, Loss: 2.3028
VGG-16 - Epoch 7, Loss: 2.3027
VGG-16 - Epoch 8, Loss: 2.3027
VGG-16 - Epoch 9, Loss: 2.3028
VGG-16 - Epoch 10, Loss: 2.3027
Training ResNet18...
ResNet18 - Epoch 1, Loss: 1.3502
ResNet18 - Epoch 2, Loss: 0.8229
ResNet18 - Epoch 3, Loss: 0.6201
ResNet18 - Epoch 4, Loss: 0.4930
ResNet18 - Epoch 5, Loss: 0.3960
ResNet18 - Epoch 6, Loss: 0.3154
ResNet18 - Epoch 7, Loss: 0.2412
ResNet18 - Epoch 8, Loss: 0.1716
ResNet18 - Epoch 9, Loss: 0.1255
ResNet18 - Epoch 10, Loss: 0.0966
Training EfficientNet-B0...
EfficientNet-B0 - Epoch 1, Loss: 1.4462
EfficientNet-B0 - Epoch 2, Loss: 0.8668
EfficientNet-B0 - Epoch 3, Loss: 0.6305
EfficientNet-B0 - Epoch 4, Loss: 0.5064
EfficientNet-B0 - Epoch 5, Loss: 0.4180
EfficientNet-B0 - Epoch 6, Loss: 0.3491
EfficientNet-B0 - Epoch 7, Loss: 0.2987
EfficientN

In [36]:
torch.save(model_A,"model_A.pth")
torch.save(model_B,"model_B.pth")
torch.save(model_C,"model_C.pth")

In [4]:
model_A=torch.load("model_A.pth", weights_only=False)
model_B=torch.load("model_B.pth", weights_only=False)
model_C=torch.load("model_C.pth", weights_only=False)
model_A.eval()
model_B.eval()
model_C.eval()
print("Models loaded successfully")

Models loaded successfully


## **5. Evaluate Model Accuracy on Clean Test Images**

In [30]:

# Evaluation function
def test_model(model, dataloader, model_name):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    print(f'{model_name} Accuracy on clean test images: {accuracy:.2f}%')
    return accuracy

clean_accuracy_A = test_model(model_A, testloader, "VGG-16")
clean_accuracy_B = test_model(model_B, testloader, "ResNet18")
clean_accuracy_C = test_model(model_C, testloader, "EfficientNet-B0")


VGG-16 Accuracy on clean test images: 10.00%
ResNet18 Accuracy on clean test images: 83.33%
EfficientNet-B0 Accuracy on clean test images: 87.77%


## **6. Apply FGSM Attack and Test Transferability**
## 6.1 Applying attack to Model A (VGG-16)

In [7]:
# Function to generate adversarial examples for Model X
def generate_adversarial_examples(model, attack):
    model.eval()
    adv_examples = []
    original_labels = []
    
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)

        adv_images = attack(images, labels).detach().cpu()
        torch.cuda.empty_cache()
        
        adv_examples.append(adv_images)
        original_labels.append(labels.cpu())

    return torch.cat(adv_examples), torch.cat(original_labels)

In [8]:
import gc
torch.cuda.empty_cache()
gc.collect()

9

In [9]:
# Apply FGSM attack to Model A (VGG-16)
fgsm_attack = torchattacks.FGSM(model_A, eps=0.03)
adv_examples_A, labels_A = generate_adversarial_examples(model_A, fgsm_attack)

In [10]:
# Function to evaluate adversarial success rate on different models
def evaluate_transferability(model, adv_examples, labels, model_name):
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for i in range(0, len(adv_examples), 16):  # Process batch of 16 images at a time
            batch_images = adv_examples[i:i+16].to(device)
            batch_labels = labels[i:i+16].to(device)

            outputs = model(batch_images)
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == batch_labels).sum().item()
            total += batch_labels.size(0)
            
            torch.cuda.empty_cache()  # Free up memory

    transfer_success_rate = 100 * (1 - correct / total)
    print(f'Transfer Success Rate on {model_name}: {transfer_success_rate:.2f}%')
    return transfer_success_rate

In [11]:
transfer_A_to_A = evaluate_transferability(model_A, adv_examples_A, labels_A, "VGG-16")
transfer_A_to_B = evaluate_transferability(model_B, adv_examples_A, labels_A, "ResNet18")
transfer_A_to_C = evaluate_transferability(model_C, adv_examples_A, labels_A, "EfficientNet-B0")

Transfer Success Rate on VGG-16: 90.00%
Transfer Success Rate on ResNet18: 52.03%
Transfer Success Rate on EfficientNet-B0: 50.72%


In [12]:
# Create transferability matrix
transfer_matrix = pd.DataFrame({
    "VGG-16 (A)": [transfer_A_to_A, transfer_A_to_B, transfer_A_to_C],
    "ResNet18 (B)": ["-", "-", "-"],  
    "EfficientNet-B0 (C)": ["-", "-", "-"]
}, index=["From VGG-16 (A)", "From ResNet18 (B)", "From EfficientNet-B0 (C)"])

print("Transferability Matrix:")
print(transfer_matrix)

Transferability Matrix:
                          VGG-16 (A) ResNet18 (B) EfficientNet-B0 (C)
From VGG-16 (A)                90.00            -                   -
From ResNet18 (B)              52.03            -                   -
From EfficientNet-B0 (C)       50.72            -                   -


## 6.2 Applying attack to Model B (ResNet18)

In [14]:
torch.cuda.empty_cache()
gc.collect()

197

In [15]:
# Apply FGSM attack to Model B (ResNet18)
fgsm_attack = torchattacks.FGSM(model_B, eps=0.03)
adv_examples_B, labels_B = generate_adversarial_examples(model_B, fgsm_attack)

In [17]:
transfer_B_to_A = evaluate_transferability(model_A, adv_examples_B, labels_B, "VGG-16")
transfer_B_to_B = evaluate_transferability(model_B, adv_examples_B, labels_B, "ResNet18")
transfer_B_to_C = evaluate_transferability(model_C, adv_examples_B, labels_B, "EfficientNet-B0")

Transfer Success Rate on VGG-16: 90.00%
Transfer Success Rate on ResNet18: 89.22%
Transfer Success Rate on EfficientNet-B0: 68.24%


In [18]:
# Create transferability matrix
transfer_matrix = pd.DataFrame({
    "VGG-16 (A)": [transfer_A_to_A, transfer_A_to_B, transfer_A_to_C],
    "ResNet18 (B)": [transfer_B_to_A, transfer_B_to_B, transfer_B_to_C],  
    "EfficientNet-B0 (C)": ["-", "-", "-"]
}, index=["From VGG-16 (A)", "From ResNet18 (B)", "From EfficientNet-B0 (C)"])

print("Transferability Matrix:")
print(transfer_matrix)

Transferability Matrix:
                          VGG-16 (A)  ResNet18 (B) EfficientNet-B0 (C)
From VGG-16 (A)                90.00         90.00                   -
From ResNet18 (B)              52.03         89.22                   -
From EfficientNet-B0 (C)       50.72         68.24                   -


## 6.3 Applying attack to Model C (EfficientNet-B0)

In [19]:
torch.cuda.empty_cache()
gc.collect()

463

In [20]:
# Apply FGSM attack to Model C (EfficientNet-B0)
fgsm_attack = torchattacks.FGSM(model_C, eps=0.03)
adv_examples_C, labels_C = generate_adversarial_examples(model_C, fgsm_attack)

In [21]:
transfer_C_to_A = evaluate_transferability(model_A, adv_examples_C, labels_C, "VGG-16")
transfer_C_to_B = evaluate_transferability(model_B, adv_examples_C, labels_C, "ResNet18")
transfer_C_to_C = evaluate_transferability(model_C, adv_examples_C, labels_C, "EfficientNet-B0")

Transfer Success Rate on VGG-16: 90.00%
Transfer Success Rate on ResNet18: 65.31%
Transfer Success Rate on EfficientNet-B0: 90.62%


In [22]:
# Create transferability matrix
transfer_matrix = pd.DataFrame({
    "VGG-16 (A)": [transfer_A_to_A, transfer_A_to_B, transfer_A_to_C],
    "ResNet18 (B)": [transfer_B_to_A, transfer_B_to_B, transfer_B_to_C],  
    "EfficientNet-B0 (C)": [transfer_C_to_A, transfer_C_to_B, transfer_C_to_C]
}, index=["From VGG-16 (A)", "From ResNet18 (B)", "From EfficientNet-B0 (C)"])

print("Transferability Matrix:")
print(transfer_matrix)

Transferability Matrix:
                          VGG-16 (A)  ResNet18 (B)  EfficientNet-B0 (C)
From VGG-16 (A)                90.00         90.00                90.00
From ResNet18 (B)              52.03         89.22                65.31
From EfficientNet-B0 (C)       50.72         68.24                90.62


## **7. Final Transferability Summary Table**

In [23]:
print("Transferability Matrix:")
print(transfer_matrix)

Transferability Matrix:
                          VGG-16 (A)  ResNet18 (B)  EfficientNet-B0 (C)
From VGG-16 (A)                90.00         90.00                90.00
From ResNet18 (B)              52.03         89.22                65.31
From EfficientNet-B0 (C)       50.72         68.24                90.62
